In [9]:
import pandas as pd

train = pd.read_csv('../../v1-no_special-no_hanja-polluted_lv/train.csv').drop(columns=['Unnamed: 0'])

target = train[(train['polluted_lv'] > 0.3)]
print(len(target))
target.sample(5)

938


,ID,text,target,polluted_lv,no_hanja
879,ynat-v1_train_00879,세p P학jE 산l 모인6…한국과t영)3a 개최,4,0.360000,세p P학jE 산l 모인6한국과t영3a 개최
2519,ynat-v1_train_02519,코z나0:i9규직Zj정규직 E문R비교,3,0.421053,코z나0i9규직Zj정규직 E문R비교
872,ynat-v1_train_00872,@대통령 Jk家 의혹 계기로pD벌WBF은닉XTq정.준,2,0.370370,대통령 Jk가 의혹 계기로pD벌WBF은닉XTq정준
364,ynat-v1_train_00364,"h시9게` 김-범호 6""상대Jxn기:L탄a바B인if2BX:",1,0.518519,h시9게 김범호 6상대Jxn기L탄a바B인if2BX
2145,ynat-v1_train_02145,D f한sM하3+미F 싱!송m이터'+시,0,0.411765,D f한sM하3미F 싱송m이터시


### 또 라마~^^

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

In [22]:
import json
from tqdm.notebook import tqdm

with open('key_maps.json', 'r', encoding='utf-8') as f:
    key_maps = json.load(f)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

train['new_text'] = None
for idx, key in tqdm(enumerate(key_maps.values()), total=len(key_maps), position=0):
    PROMPT = f'''당신은 기사 제목을 복원하는 전문가입니다.
    1. 키워드는 '{key}'입니다.
    2. 어색한 문맥을 키워드를 참고하여 올바르게 고치세요.
    3. 복원이 안 되거나 일부만 가능할 경우, 해당 키워드를 포함한 형태로 대체하세요.'''

    fewshot = [
        {"role": "user", "content": "해외로밍 m금폭탄 n동차단 더 빨진다"},
        {"role": "assistant", "content": "해외로밍 요금 폭탄 자동차단 더 빨라진다"},
        # {"role": "user", "content": "갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩"},
        # {"role": "assistant", "content": "갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩"},
        {"role": "user", "content": "코z나0i9규직Zj정규직 E문R비교"},
        {"role": "assistant", "content": f"{key}코로나 정규직 비정규직 E문R비교"},
        {"role": "user", "content": "정i 파1 미사z KT 이용기간 2e 단 Q분종U2보"},
        {"role": "assistant", "content": f"{key} 정i 파1 미사z KT 이용기간 2e 단 Q분종U2보"}
    ]

    key_target = target[target['target']==idx]
    for data in tqdm(key_target.iterrows(), desc=f'label={idx}', total=len(key_target), position=1, leave=False):
        data = data[1]
        
        messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": data['no_hanja']}]
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1024,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

        result = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        # print(data['no_hanja'])
        # print(result)
        # print()
        train.loc[train['ID']==data['ID'], 'new_text'] = result

# train.head()
train[train['polluted_lv']>0.3].head()

  0%|          | 0/7 [00:00<?, ?it/s]

label=0:   0%|          | 0/130 [00:00<?, ?it/s]

label=1:   0%|          | 0/133 [00:00<?, ?it/s]

label=2:   0%|          | 0/118 [00:00<?, ?it/s]

label=3:   0%|          | 0/124 [00:00<?, ?it/s]

label=4:   0%|          | 0/160 [00:00<?, ?it/s]

label=5:   0%|          | 0/155 [00:00<?, ?it/s]

label=6:   0%|          | 0/118 [00:00<?, ?it/s]

,ID,text,target,polluted_lv,no_hanja,new_text
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,0.344828,정i 파1 미사z KT 이용기간 2e 단 Q분종U2보,기술 정파 1 미사일 KT 이용기간 2e 단 Q분종 U2 보
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,0.500000,K찰국DLwo 로L3한N 회장 2 T0송,사회 K찰국DLwo 로L3한N 회장 2 T0송
10,ynat-v1_train_00010,oi 매력 R모h츠a열#w3약 >l·주가 고Q/진,5,0.391304,oi 매력 R모h츠a열w3약 l주가 고Q진,경제 oi 매력 R모h츠a열w3약 주가 고Q진
13,ynat-v1_train_00013,아이`XSI수리0* b대`…맥3 디dF레< 41/',4,0.476190,아이XSI수리0 b대맥3 디dF레 41,기술 AI XSI 수리 0 대 맥 3 디드 F 레 41
14,ynat-v1_train_00014,"문/인 당2 4nS 민관2동7사위 /""X보 철거tt",2,0.360000,문인 당2 4nS 민관2동7사위 X보 철거tt,"문인 당2 4nS 민관2동7사위 X보 철거tt (문인 당 2명, 민관 2동 7사위 ..."


In [ ]:
for data in train.iterrows():
    data = data[1]
    if data['new_text'] is not None:
        train.loc[train['ID']==data['ID'], 'text'] = data['new_text']

train = train.drop(columns=['new_text', 'no_hanja'])
train.to_csv('../../v3-label_txt/train.csv', index=False)

### MLM 모델로 역번역

### Tokenizer로 encoding했다가 decoding

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

for data in train.iterrows():
    data = data[1]
    tokenized = tokenizer(data['no_hanja'])
    print(f"{data['no_hanja']}\t\t\t{tokenizer.decode(tokenized['input_ids'], skip_special_tokens=True)}")

정i 파1 미사z KT 이용기간 2e 단 Q분종U2보			정i 파1 미사z KT 이용기간 2e 단 Q분종U2보
K찰국DLwo 로L3한N 회장 2 T0송			K찰국DLwo 로L3한N 회장 2 T0송
m 김정 자주통일 새r열1나가야1보			m 김정 자주통일 새r열1나가야1보
갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩			갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩
pI미대선I앞두고 R2fr단 발 비해 감시 강화			pI미대선I앞두고 R2fr단 발 비해 감시 강화
미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다			미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다
프로야구롯TKIAs광주 경기 y천취소			프로야구롯TKIAs광주 경기 y천취소
아가메즈 33득점 우리카드 KB손해보험 완파3위 굳			아가메즈 33득점 우리카드 KB손해보험 완파3위 굳
박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합			박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합
듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감			듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감
oi 매력 R모h츠a열w3약 l주가 고Q진			oi 매력 R모h츠a열w3약 l주가 고Q진
NH투자 1월 옵션 만기일 매도 우세			NH투자 1월 옵션 만기일 매도 우세
황총리 각 부처 비상대비태세 철저히 강구해야			황총리 각 부처 비상대비태세 철저히 강구해야
아이XSI수리0 b대맥3 디dF레 41			아이XSI수리0 b대맥3 디dF레 41
문인 당2 4nS 민관2동7사위 X보 철거tt			문인 당2 4nS 민관2동7사위 X보 철거tt
게시판 KISA 박민정 책임연구원 APTLD 이사 선출			게시판 KISA 박민정 책임연구원 APTLD 이사 선출
공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예			공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예
월세 전환에 늘어나는 주거비 부담작년 역대 최고치			월세 전환에 늘어나는 주거비 부담작년 역대 최고치
개R전 연w정연H 작가			개R전

### 형태소 분석기

In [ ]:
from konlpy.tag import Mecab, Hannanum, Kkma, Komoran, Okt

mecab_tokenizer = Mecab().morphs
hannanum_tokenizer = Hannanum().morphs
kkma_tokenizer = Kkma().morphs
komoran_tokenizer = Komoran().morphs
okt_tokenizer = Okt().morphs

['정i', ':파1', '미사z', 'KT', '(', '이용기간', '2e', '단', ']', 'Q분종U2보']


In [25]:
for data in train.iterrows():
    data = data[1]
    mecab_tokenized = mecab_tokenizer(data['no_hanja'])
    hannanum_tokenized = hannanum_tokenizer(data['no_hanja'])
    kkma_tokenized = kkma_tokenizer(data['no_hanja'])
    komoran_tokenized = komoran_tokenizer(data['no_hanja'])
    okt_tokenized = okt_tokenizer(data['no_hanja'])

    print(data['no_hanja'])
    print(mecab_tokenized)
    print(hannanum_tokenized)
    print(kkma_tokenized)
    print(komoran_tokenized)
    print(okt_tokenized)
    print()

정i 파1 미사z KT 이용기간 2e 단 Q분종U2보
['정', 'i', '파', '1', '미사', 'z', 'KT', '이용', '기간', '2', 'e', '단', 'Q', '분종', 'U', '2', '보']
['정i', '파1', '미사z', 'KT', '이용기간', '2e', '단', 'Q분종U2보']
['정', 'i', '파', '1', '미', '사', 'z', 'KT', '이용', '기간', '2', 'e', '닿', 'ㄴ', 'Q', '분종', 'U', '2', '보']
['정', 'i', '파', '1', '미사', 'z', 'KT', '이용', '기간', '2', 'e', '단', 'Q', '분', '종', 'U2', '보']
['정', 'i', '파', '1', '미사', 'z', 'KT', '이용기', '간', '2', 'e', '단', 'Q', '분종', 'U', '2', '보']

K찰국DLwo 로L3한N 회장 2 T0송
['K', '찰', '국', 'DLwo', '로', 'L', '3', '한', 'N', '회장', '2', 'T', '0', '송']
['K찰국DLwo', '로L3한N', '회장', '2', 'T0송']
['K', '차', 'ㄹ', '국', 'DLwo', '로', 'L', '3', '한', 'N', '회장', '2', 'T', '0', '송']
['K', '차', 'ㄹ', '국', 'DLwo', '로', 'L3', '한', 'N', '회장', '2', 'T', '0', '송']
['K', '찰국', 'DLwo', '로', 'L', '3', '한', 'N', '회장', '2', 'T', '0', '송']

m 김정 자주통일 새r열1나가야1보
['m', '김정', '자주', '통일', '새', 'r', '열', '1', '나가', '야', '1', '보']
['m', '김정', '자주통', '이', 'ㄹ', '새r열1나가야1보']
['m', '김', '정', '자주', '통일', '새', 'r', '열', '1', '